In [1]:
import pandas as pd
import numpy as np
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
articoli = pd.read_csv("df.csv", sep=";")
articoli.head()

Unnamed: 0                                ID  \
0           0  c812e0a3418d624ae60df3ec9e19d184   
1           1  0e2ef6bd7619374e0531d1551ab2141e   
2           2  8602d8791a1754506303dcfd9a7b79e3   
3           3  2157ac6d12e063b6a38c5c0491e0b322   
4           4  89390733d3f9e1f30bae53de5e2d8a04   

                                              Titolo Abstract  \
0                       pandemic drone spot infected      NaN   
1                     census data anonymous expected      NaN   
2  common pandemic scams highlighted domain name ...      NaN   
3   questions rapidly unfolding future smart fabrics      NaN   
4             types robots playing vital roles covid      NaN   

                                               Testo  \
0  spotted current climate major problems control...   
1  census multiple ads running online offline med...   
2  coronavirus lockdowns slowly starting unwind a...   
3  soon clothes able monitor vital signs analyze ...   
4  perfect storm disruptions supply demand workpl...   

                             Tech  \
0  Cobots and Human Hybridization   
1  Privacy and Security by design   
2        Security & Cyber Warfare   
3        Workforce Transformation   
4       New Normal after COVID-19   

                                                 Txt  
0  pandemic drone spot infectedspotted current cl...  
1  census data anonymous expectedcensus multiple ...  
2  common pandemic scams highlighted domain name ...  
3  questions rapidly unfolding future smart fabri...  
4  types robots playing vital roles covidperfect ...

In [3]:
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = nltk.corpus.stopwords.words('english')
my_stopwords = ['also','said', 'one', 'may', 'polls', 'sorry', 'like', 'use', 'many', 'make', 'could', 
                'even', 'says', 'new', 'us', 'would', 'time', 'companies', 'moment', 'way', 'using', 'two',
               'across', 'well', 'world']

stop_words.extend(my_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\edoardo.esposito\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
import re

corpus = articoli['Txt']
cv=CountVectorizer(max_df=0.8,stop_words=stop_words, max_features=10000, ngram_range=(1,3))
X=cv.fit_transform(corpus)

In [5]:
# TODO - List all and remove stopwords
list(cv.vocabulary_.keys())[:10]

['pandemic',
 'drone',
 'spot',
 'current',
 'climate',
 'major',
 'problems',
 'controlling',
 'covid',
 'knowing']

In [6]:
# convert text to features
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(articoli['Tech'])

MultiLabelBinarizer(classes=None, sparse_output=False)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
#tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
tfidfconverter = TfidfVectorizer(min_df=5, max_df=0.7, stop_words=stop_words)
X = tfidfconverter.fit_transform(articoli['Txt']).toarray()

In [8]:
# https://stackabuse.com/text-classification-with-python-and-scikit-learn/
# INTERESTING FOR SAVING MODEL
from pandas import Series
y = Series(articoli['Tech']).values

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [10]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train) 

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [11]:
y_pred = classifier.predict(X_test)

In [12]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average="micro")

0.43859649122807015

In [48]:
startup = pd.read_csv('startup - startup.csv', sep=",")
del(startup['Unnamed: 0'])
startup.head()

Nome                                               Text
0   1CONTROL  control a startup born from idea to revolution...
1   200CROWD  two hundred crowd an equity portal crowdfundin...
2  2MANYAPPS  manyapps manyapps a web marketing agency focus...
3  4 GIFTERS  gifters gifters a and gifting that allows you ...
4      ABIBY  abiby abiby a startup that select the best pro...

In [49]:
p_trends = []
for row in startup.itertuples():
    test_data = np.array([row[2]])
    testVectorizerArray = tfidfconverter.transform(test_data).toarray()
    p_trends.append(classifier.predict(testVectorizerArray)[0])

In [50]:
p_series = pd.Series(p_trends)

In [51]:
startup = pd.concat([startup,p_series], axis=1)

In [52]:
startup.head()

Nome                                               Text  \
0   1CONTROL  control a startup born from idea to revolution...   
1   200CROWD  two hundred crowd an equity portal crowdfundin...   
2  2MANYAPPS  manyapps manyapps a web marketing agency focus...   
3  4 GIFTERS  gifters gifters a and gifting that allows you ...   
4      ABIBY  abiby abiby a startup that select the best pro...   

                               0  
0  3D Printing and New Materials  
1             Future of Mobility  
2      New Normal after COVID-19  
3      New Normal after COVID-19  
4                    Health Tech

In [53]:
#startup.to_csv("st1.csv", sep=";")